# Tarea 3: Bases de datos 
### <span style="color:blue"> Eduardo Alejandro Arrieta Donato</span> 
## índice
+ <a href="#Presentacion" style="color:pink">Presentación</a>
+ <a href="#Ejercicio_1" style="color:pink">Ejercicio 1</a>
+ <a href="#Ejercicio_2" style="color:pink">Ejercicio 2</a>
+ <a href="#Ejercicio_3" style="color:pink">Ejercicio 3</a>
+ <a href="#Ejercicio_4" style="color:pink">Ejercicio 4</a>
+ <a href="#DONTCLICKME" style="color:pink">DON´T CLICKME</a>

<a id="Presentacion"></a>
## <span style="color:yellow"> Presentación</span>
Hola, esta es mi tarea3 del curso.

Cada ejercicio resulto viene con sus indicaciones generales y libertades que se tomaron.

Cada celda ocupada para resolver tendrá sus respectiva documentación de código.

Todos los $INPUTS$ serán *in-cell*, pues considero sucio hacerlo con `input()`, los inputos se destacaran con la leyenda <span style="color:orange"> *'''INPUT'''*</span>

<a id="Ejercicio_1"></a>

## <span style="color:yellow"> Ejercicio_1</span>
Arreglar el código para que solo muestre las cominaciones únicas

<span style="color:green">Cambié un for normal por una comprehention list.</span>

In [1]:
import Bio.PDB.PDBParser

'''INPUT'''
file = '1kcw.pdb'

name = file.replace('.pdb', '').upper()

parser = Bio.PDB.PDBParser(QUIET=True)
structure = parser.get_structure(name, file)
model = structure[0]
achain = model['A']

#Voy por cada residuo hasta encontrar CYS
cysresidues = [residue for residue in achain if residue.get_resname() == 'CYS']



# Veo todos los residuos y sus combinaciones. Como vimos anteriormente, al restar átomos nos da la distancia
# Guardaremos los repetidos en set para llevar un control
repertidos = set()
for c1 in cysresidues:
    c1index = c1.get_id()[1]
    for c2 in cysresidues:
        c2index = c2.get_id()[1]
        
        # Si son el mismo o ys se encuntra en los repetidos, entonces lo saltará
        if c1index == c2index or c2index in repertidos:
            continue
            
        #Estoy enteresada en el átomo de SG "sulfur-gamma"
        if (c1['SG'] - c2['SG']) < 8.0:
            print("possible di-sulfide bond:", end=" ")
            print("Cys", c1index, "-" , end=" ")
            print("Cys", c2index, end=" ")
            print(round(c1['SG'] - c2['SG'], 2))
            
    # Lo agrega a los repetidos
    repertidos.add(c1index)

# Pa' que o guardo
del repertidos

possible di-sulfide bond: Cys 155 - Cys 181 2.0
possible di-sulfide bond: Cys 257 - Cys 338 2.03
possible di-sulfide bond: Cys 515 - Cys 541 2.02
possible di-sulfide bond: Cys 618 - Cys 699 2.03
possible di-sulfide bond: Cys 855 - Cys 881 2.02


<a id="Ejercicio_2"></a>

## <span style="color:yellow"> Ejercicio_2</span>
BS3 es un agente entrecruzante impermeable a la membrana plasmática. Sirve para el estudio de interacciones de proteínas de membrana plasmática con residuos de lisina accesibles desde el medio extracelular.

+ Encuentren los pares de lisinas que podrían ser unidas utilizando BS3

La distancia suele ser menor a 12 Å

In [2]:
from Bio.PDB.Polypeptide import is_aa
import Bio.PDB.PDBParser

'''INPUT'''
file = '1ce0.pdb'

name = file.replace('.pdb', '').upper()

# Genero el parser
parser = Bio.PDB.PDBParser(QUIET=True)
# Extraigo la estructura desde el documento
structure = parser.get_structure(name, file)

# Obtengo ambas cadenas
achain = structure[0]['A']
bchain = structure[0]['B']

#Obtengo la lista de vecinos en la cadena B si son lisinas
bchainNZ = [r['NZ'] for r in bchain if is_aa(r) and r.get_resname() == 'LYS']
neighbors = Bio.PDB.NeighborSearch(bchainNZ)

# Recorro la A
for res1 in achain:
    # Unicamente si son lisisnas
    if is_aa(res1) and res1.get_resname() == 'LYS':
        # Obtengo su átomo colita y demás info
        r1_NZ = res1['NZ']
        r1_ind = res1.get_id()[1]
        r1_sym = res1.get_resname()
        # Recorro su relación con la lista de vecinos de B con un radio menor a 12
        for r2_NZ in neighbors.search(r1_NZ.get_coord(), 12.0):
            # Obtengo el residuo y demás info
            res2 = r2_NZ.get_parent()
            r2_ind = res2.get_id()[1]
            r2_sym = res2.get_resname()
            
            # Imprimo resultados
            print(f'{r1_sym} {r1_ind} en A y {r2_sym} {r2_ind} en B están {r1_NZ - r2_NZ}')

LYS 12 en A y LYS 7 en B están 11.294949531555176


<a id="Ejercicio_3"></a>

## <span style="color:yellow"> Ejercicio_3</span>
Utilizando Entrez.einfo y Entrez.read, imprime la descripción de:
    
    
+ field **'ECNO'**
+ link **'protein_protein_small_genome'**
    
    
de la base de datos 'protein'

In [1]:
from Bio import Entrez

# Me identifico
Entrez.email = 'earrieta@lcg.unam.mx'

# Pido la info de la base de datos protein
with Entrez.einfo(db='protein') as handle:
    # Recolecto su info
    record = Entrez.read(handle)

    # Recorro cada field
    for field in record['DbInfo']['FieldList']:
        # Imprimo solo el que me piden
        if field['Name'] == 'ECNO':
            print(field['Description'])

    # Recorro cada link
    for link in record['DbInfo']['LinkList']:
        # Imprimo solo la que me piden
        if link['Name'] =='protein_protein_small_genome':
            print (link['Description'])

EC number for enzyme or CAS registry number
All proteins from this genome


<a id="Ejercicio_4"></a>

## <span style="color:yellow"> Ejercicio_4</span>
Automaticen para buscar los siguientes genes en la base de datos nucleotide y guarden las secuencias de las proteínas en un archivo fasta: 
    
    
    Organismo: Notoryctes typhlops             Genes: PNOC y RUNX2
    Organismo: Cypripedium guttatum            Genes: matK
    Organismo: Hepatitis B virus               Genes: X                     Aislado: migrant 467

<span style="color:green">Como la tarea lo pide automatizado, quise que las especificaciones vengan de un diccionario (en esta ocasion la construcción es manual, pero se entiende que se va a recuperar de algún proceso anterior).</span>

In [2]:
from Bio import Entrez

'''INPUT'''
# Genero el diciconario de la búsqueda
# Las key será los organismos
# Los values una tuplas, donde el primer elemento es una lista de genes y el segundo con caracteristica de la búsqueda
busqueda = {'Notoryctes typhlops':(['PNOC', 'RUNX2']),
            'Cypripedium guttatum':(['matK']),
            'Hepatitis B virus':(['X'], 'migrant 467')
           }
# Aqui se especifica que característica es
extraTupla = '[Isolate]'

In [3]:
# Una lista vacía que contre los ID para Entrez
UIDs = list()

# Recorro el diccionario con key y value
for organism, more in busqueda.items():
    
    # Si es el caso complejo...
    if type(more) == tuple:
        # Recorro cada gene
        for gene in more[0]:
            # Genero cada termino de búsqueda anezando la característica extra
            termino = '{}[Organism] AND {}[Gene] AND {}{}'.format(organism, gene, more[1], extraTupla)
            
            # Aqui hago la consulta para recuperar el UID
            with Entrez.esearch(db='nucleotide', term= termino) as handle:
                # Recupero el ID del primer resultado (solo será una en total por como estpa planteada la actividad)
                UIDs.append(Entrez.read(handle)['IdList'][0])
    
    # Si el caso simple
    else:
        # Recorro cada gen
        for gene in more:
            # Gnero un término de busqyeda por cada gen
            termino = '{}[Organism] AND {}[Gene]'.format(organism, gene)
            
            # Hago la consulta
            with Entrez.esearch(db='nucleotide', term= termino) as handle:
                # Recupero el ID del primer resultado
                UIDs.append(Entrez.read(handle)['IdList'][0])

# Vemos la lista           
UIDs

['347453667', '1183154904', '635173148', '299836331']

In [4]:
from Bio import SeqIO

# Aqui recuperaremos el fasta de cada UID
for UID in UIDs:
    # Hago la consulta, le pido fasta y creo un archivo vacío .fa por cada UID
    with Entrez.efetch(db="nucleotide", id=UID, rettype='fasta', retmode="text") as handle, \
    open(UID + '.fa', 'w') as output:
        
            # Leo lo obtenido y lo guardo en el archivo
            SeqIO.write(SeqIO.read(handle, 'fasta'), output, 'fasta')

<a id="DONTCLICKME"></a>
## <span style="color:yellow"> CHOMP!</span>
![Te dije que no le dieras clic](https://m.gifanimados.com/Gifs-Animales/Animaciones-Animales-Marinos/Peces/Tiburones/Boca-Tiburon-69162.gif)